In [1]:
import seaborn as sns

%config InlineBackend.figure_format = 'retina'

sns.set_theme(style='darkgrid')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

df = pd.read_csv("data/wta.csv")

df["Player_1"] = df["Player_1"].str.strip().str.lower()
df["Player_2"] = df["Player_2"].str.strip().str.lower()
df["Winner"] = df["Winner"].str.strip().str.lower()
df["Winner"] = df["Winner"] == df["Player_1"]

unique_players = pd.unique(df[["Player_1", "Player_2"]].values.ravel())
player_to_id = {player: idx for idx, player in enumerate(unique_players)}

df["Player_1"] = df["Player_1"].map(player_to_id)
df["Player_2"] = df["Player_2"].map(player_to_id)

df

/var/folders/60/myb807dx2ksbq9r6r4vtxbf00000gn/T/ipykernel_29161/3290745162.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/wta.csv")


,Tournament,Date,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
0,ASB Classic,2007-01-01 00:00:00,Outdoor,Hard,1st Round,3,0,1,True,81,272,332,90,1.33,3.00,6-1 6-1
1,ASB Classic,2007-01-01 00:00:00,Outdoor,Hard,1st Round,3,2,3,False,16,59,1000,401,1.22,3.75,1-6 7-6 2-6
2,ASB Classic,2007-01-01 00:00:00,Outdoor,Hard,1st Round,3,4,5,True,56,84,418,324,1.72,2.00,6-1 6-1
3,ASB Classic,2007-01-01 00:00:00,Outdoor,Hard,1st Round,3,6,7,False,57,70,405,365,1.83,1.83,5-7 2-6
4,ASB Classic,2007-01-01 00:00:00,Outdoor,Hard,1st Round,3,8,9,True,18,180,951,152,1.16,4.50,7-6 6-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41892,Charleston Open,2025-04-04 00:00:00,Outdoor,Clay,Quarterfinals,3,778,1017,True,26,8,1778,4135,2.75,1.44,6-1 6-4
41893,Charleston Open,2025-04-04 00:00:00,Outdoor,Clay,Quarterfinals,3,832,745,False,33,44,1524,1263,2.1,1.73,4-6 3-6
41894,Charleston Open,2025-04-04 00:00:00,Outdoor,Clay,Quarterfinals,3,843,930,True,16,11,2454,3749,1.8,2.00,7-5 7-6
41895,Charleston Open,2025-04-05 00:00:00,Outdoor,Clay,Semifinals,3,778,574,False,26,4,1778,5796,2.5,1.53,2-6 6-2 5-7


In [3]:
import numpy as np

p = unique_players.shape[0]
N = np.zeros((p, p), dtype=int)
S = np.zeros((p, p), dtype=int)
for i, row in df.iterrows():
    winner = row["Winner"]
    player_1 = row["Player_1"]
    player_2 = row["Player_2"]
    N[player_1, player_2] += 1
    S[player_1, player_2] += winner
    S[player_2, player_1] += 1 - winner

N

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1224, 1224))

In [4]:
def connected_components_sizes(N):
    # number of nodes
    p = N.shape[0]
    # initialize Union-Find parent array
    parent = list(range(p))

    def find(x):
        # path compression find
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(x, y):
        rx = find(x)
        ry = find(y)
        if rx != ry:
            parent[ry] = rx

    # treat edge existence as undirected: if either N[i,j] or N[j,i] is nonzero, they are connected.
    for i in range(p):
        for j in range(i + 1, p):
            if N[i, j] != 0 or N[j, i] != 0:
                union(i, j)

    # count the size of each connected component
    comp_sizes = {}
    for i in range(p):
        root = find(i)
        comp_sizes[root] = comp_sizes.get(root, 0) + 1

    return list(comp_sizes.values())


# Calculate and print the sizes of the connected components.
component_sizes = connected_components_sizes(N)
print("Connected component sizes:", component_sizes)

Connected component sizes: [1224]


In [5]:
from data import GaoData, is_connected


data = GaoData(p=p, edge_prob=0, L=0, v_range=(0, 1), generate=False)

data.N, data.S = N, S
data.v = np.random.normal(0, 1, p)
data.N

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1224, 1224))

In [6]:
from estimate import calc_v_g_tilda_sum_reg

data.edge_prob = 1
data.L = 1

v_g_tilda = calc_v_g_tilda_sum_reg(data, 1000)
v_g_tilda

/Users/honeybadger/hse/btl/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


iter 0, grad norm 1177.3914462843054, f 62975.15492359346, x [-0.30545661 -0.90162781  0.21680286 ...  0.00140308 -1.22326976
 -1.28171232]
iter 1, grad norm 1069.204902249418, f 82062.00638689045, x [-1.80801962 -1.03714982  0.01680286 ... -0.0703282  -1.24598127
 -1.38171232]
iter 2, grad norm 1030.4259806718503, f 99772.19909764724, x [-3.39782569 -1.20552072 -0.18319714 ... -0.16186193 -1.26956474
 -1.48171232]
iter 3, grad norm 1009.3479896006312, f 115599.87919511086, x [-4.95576944 -1.39547636 -0.38319714 ... -0.25970395 -1.29439468
 -1.58171232]
iter 4, grad norm 1012.2392423186723, f 138443.4290909932, x [-6.51559717 -1.59271191 -0.58319714 ... -0.35917208 -1.32061225
 -1.68171232]
iter 5, grad norm 1006.3805090923328, f 155409.10939592804, x [-8.0670997  -1.79198935 -0.78319714 ... -0.4590415  -1.34827125
 -1.78171232]
iter 6, grad norm 1000.0243980642935, f 176648.34108647145, x [-9.61194868 -1.99180145 -0.98319714 ... -0.55900941 -1.37739813
 -1.88171232]
iter 7, grad norm 

/Users/honeybadger/hse/btl/btl_func.py:10: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-v))
/Users/honeybadger/hse/btl/btl_func.py:6: RuntimeWarning: overflow encountered in exp
  return np.log(1 + np.exp(v))
/Users/honeybadger/hse/btl/btl_func.py:75: RuntimeWarning: invalid value encountered in scalar multiply
  result += (v[j] - v[m]) * data.S[j, m] - data.N[j, m] * phi(


iter 141, grad norm 980.2800292206965, f nan, x [-212.49193555  -28.99173498  -27.98319714 ...  -14.05899893  -11.67531303
  -15.38171232]
iter 142, grad norm 974.7994403754648, f nan, x [-213.9919356   -29.19173498  -28.18319714 ...  -14.15899893  -11.76687051
  -15.48171232]
iter 143, grad norm 975.0784998649456, f nan, x [-215.49193564  -29.39173498  -28.38319714 ...  -14.25899893  -11.85849302
  -15.58171232]
iter 144, grad norm 985.0622152753205, f nan, x [-216.99193568  -29.59173498  -28.58319714 ...  -14.35899893  -11.9501796
  -15.68171232]
iter 145, grad norm 981.310423787669, f nan, x [-218.49193571  -29.79173498  -28.78319714 ...  -14.45899893  -12.04192934
  -15.78171232]
iter 146, grad norm 981.8194924828216, f nan, x [-219.99193574  -29.99173498  -28.98319714 ...  -14.55899893  -12.13374131
  -15.88171232]
iter 147, grad norm 979.3718722063481, f nan, x [-221.49193577  -30.19173498  -29.18319714 ...  -14.65899893  -12.22561463
  -15.98171232]
iter 148, grad norm 980.90028

KeyboardInterrupt: 

In [20]:
from btl_func import phi, phi_1, phi_2

print(phi(0))
print(phi_1(0))
print(phi_2(0))

print(phi(-1e12))
print(phi_1(-1e12))
print(phi_2(-1e12))

0.6931471805599453
0.5
0.25
0.0
0.0
0.0


In [ ]:
print(data.N.sum(axis=1).max())

417
